In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
pwd

'/content'

In [4]:
cd /content/gdrive/My Drive/test/Mr. Cooper/

/content/gdrive/My Drive/test/Mr. Cooper


In [6]:
!pip install catboost
!pip install contractions

     |████████████████████████████████| 284 kB 5.4 MB/s 
     |████████████████████████████████| 321 kB 40.0 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85441 sha256=e7f0af27a51dd9e7a438a93a55e08a9678fb1e193a7d1a560c1bfd6e91d0dc77
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [29]:
import re
import contractions
import spacy
import unicodedata

import os
import inflect
import nltk
import pickle
import re, string, unicodedata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, confusion_matrix
from nltk.corpus import stopwords
from catboost.utils import get_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')


In [14]:
# Label Enconding
le = LabelEncoder()

# Loading Spacy Model
nlp = spacy.load('en_core_web_sm')

In [48]:
# reading data from .csv file
data_df = pd.read_csv("data/Context.csv")
data_df.head(3)

,Text,Context/Topic
0,The eternal mystique of Goldman Sachs,Politics
1,Either you don't care enough to actually tell ...,Love
2,I am such an IDIOT.,Heavy Emotion


In [49]:
# Each feature type info
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31386 entries, 0 to 31385
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Text           31386 non-null  object
 1   Context/Topic  31386 non-null  object
dtypes: object(2)
memory usage: 490.5+ KB


In [50]:
# Size of total dataset
data_df.shape

(31386, 2)

In [51]:
# Checking if any Null value is present
data_df.isna().sum()

Text             0
Context/Topic    0
dtype: int64

In [52]:
# Checking if any Duplicate value is present
data_df.duplicated().sum()

0

In [53]:
def remove_urls(text):
    text = re.sub(r'http\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'https\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'www\S+', ' ', text, flags=re.MULTILINE)
    return (text)


def remove_bracket_words(text):
    text = re.sub("[\(\[].*?[\)\]]", " ", text)
    return text


def remove_contraction_word(text):
    text = " ".join([contractions.fix(i) for i in text.split() if len(i) > 0 and "@" not in i])
    return text


def remove_ascii_character(text):
    text = re.sub('[^.,:a-zA-Z0-9 \n\.]', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def cleaning_pipeline(text):
    cleaned_text = remove_urls(text)
    cleaned_text = remove_bracket_words(cleaned_text)
    cleaned_text = remove_contraction_word(cleaned_text)
    cleaned_text = remove_ascii_character(cleaned_text)
    cleaned_text = remove_accented_chars(text)

    return cleaned_text


In [54]:
# text = data_df['Text'][5]
# print(text)
# cleaning_pipeline(text)

Plantação de palma na Indonésia é uma dura realidade para os orangotangos


'Plantacao de palma na Indonesia e uma dura realidade para os orangotangos'

In [44]:
cleaned_text = []
count = 1
for text in data_df['Text']:
    cleaned_text.append(cleaning_pipeline(text))
    if count%10000 == 0:
        print("Records processed:\t", count)
    # break
    count = count + 1
print("Records processed:\t", count)

Records processed:	 10000
Records processed:	 20000
Records processed:	 30000
Records processed:	 31387


In [57]:
data_df = data_df.assign(cleaned_text=cleaned_text)

,Text,Context/Topic,cleaned_text
0,The eternal mystique of Goldman Sachs,Politics,The eternal mystique of Goldman Sachs
1,Either you don't care enough to actually tell ...,Love,Either you don't care enough to actually tell ...
2,I am such an IDIOT.,Heavy Emotion,I am such an IDIOT.


In [59]:
# Desctiption of each features
for col in data_df.columns:
    print("\n***********************")
    print("Columns:", col)
    print(data_df[col].describe())



***********************
Columns: Text
count                                                 31386
unique                                                31386
top       If you could hijack another broadcast today, w...
freq                                                      1
Name: Text, dtype: object

***********************
Columns: Context/Topic
count             31386
unique               11
top       Heavy Emotion
freq               3674
Name: Context/Topic, dtype: object

***********************
Columns: cleaned_text
count     31386
unique    31317
top            
freq         29
Name: cleaned_text, dtype: object


In [60]:
data_df['Context/Topic'].value_counts()

Heavy Emotion    3674
Religion         3466
Love             3229
Self             3105
Compliment       3061
Animals          2622
Health           2595
Education        2534
Joke             2476
Science          2428
Politics         2196
Name: Context/Topic, dtype: int64

In [69]:
length = []
for text in data_df['cleaned_text']:
    length.append(len(text.split()))
    
data_df['length'] = length

In [70]:
data_df['context_topic'] = le.fit_transform(data_df['Context/Topic'])
data_df.head(3)

,Text,Context/Topic,cleaned_text,length,context_topic
0,The eternal mystique of Goldman Sachs,Politics,The eternal mystique of Goldman Sachs,6,7
1,Either you don't care enough to actually tell ...,Love,Either you don't care enough to actually tell ...,93,6
2,I am such an IDIOT.,Heavy Emotion,I am such an IDIOT.,5,4


In [71]:
fig = px.histogram(data_df, x="length")
fig.show()

In [72]:
data_df['length'].describe()

count    31386.000000
mean        24.330816
std         43.529349
min          0.000000
25%          6.000000
50%         11.000000
75%         24.000000
max       1132.000000
Name: length, dtype: float64

In [73]:
topic_avg_leg = []

for each_g in data_df.groupby(['Context/Topic']):
    topic_avg_leg.append([each_g[0], np.mean(each_g[1]['length'])])
#     break
    

In [74]:
df = pd.DataFrame(topic_avg_leg, columns=['topic', 'length'])

fig = px.bar(df.sort_values(['length']), x='topic', y='length',
             color='length',
             height=400)
fig.show()

In [75]:
data_mod_df = data_df[(data_df['length'] > 0) & (data_df['length'] <= 50)]

In [76]:
data_mod_df.shape

(27797, 5)

In [77]:
data_mod_df['Context/Topic'].value_counts()

Religion         3039
Compliment       2882
Self             2877
Heavy Emotion    2769
Animals          2577
Health           2471
Love             2464
Science          2395
Education        2233
Politics         2189
Joke             1901
Name: Context/Topic, dtype: int64

In [78]:
# check class distribution
data_mod_df['Context/Topic'].value_counts(normalize = True)

Religion         0.109328
Compliment       0.103680
Self             0.103500
Heavy Emotion    0.099615
Animals          0.092708
Health           0.088894
Love             0.088643
Science          0.086160
Education        0.080332
Politics         0.078750
Joke             0.068389
Name: Context/Topic, dtype: float64

In [79]:
train_text, temp_text, train_labels, temp_labels = train_test_split(data_mod_df['Text'], data_mod_df['context_topic'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=data_mod_df['context_topic'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [80]:
print("Size of Independent Training Data", train_text.shape)
print("Size of Dependent Training Data", train_labels.shape)
print("Size of Independent Test Data", test_text.shape)
print("Size of Dependent Training Data", test_labels.shape)
print("Size of Independent Validation Data", val_text.shape)
print("Size of Dependent Validation Data", val_labels.shape)


Size of Independent Training Data (19457,)
Size of Dependent Training Data (19457,)
Size of Independent Test Data (4170,)
Size of Dependent Training Data (4170,)
Size of Independent Validation Data (4170,)
Size of Dependent Validation Data (4170,)


In [81]:
data_mod_df = data_mod_df[['Text', 'context_topic']]

df_train = data_mod_df[0:int(data_mod_df.shape[0]*0.90)]
df_test = data_mod_df[0:int(data_mod_df.shape[0]*0.10)]

In [82]:
train, valid = train_test_split(
    df_train,
    train_size=0.80,
    random_state=0,
    stratify=df_train['context_topic'])

y_train, X_train = train['context_topic'], train.drop(['context_topic'], axis=1)
y_valid, X_valid = valid['context_topic'], valid.drop(['context_topic'], axis=1)

In [83]:
X_train.shape, X_valid.shape

((20013, 1), (5004, 1))

In [84]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=['Text']
)

valid_pool = Pool(
    data=X_valid, 
    label=y_valid,
    text_features=['Text']
)

test_pool = Pool(
    data=df_test[[c for c in df_test.columns if c!='context_topic']][['Text']], 
    label=df_test['context_topic'],
    text_features=['Text']
)

In [85]:
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        task_type='GPU',
        iterations=5000,
        eval_metric='Accuracy',
        od_type='Iter',
        od_wait=500,
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=test_pool,
        verbose=100,
        plot=True,
        use_best_model=True)

In [86]:
model = fit_model(
    train_pool, valid_pool,
    learning_rate=0.35,
    tokenizers=[
        {
            'tokenizer_id': 'Sense',
            'separator_type': 'BySense',
            'lowercasing': 'True',
            'token_types':['Word', 'Number', 'SentenceBreak'],
            'sub_tokens_policy':'SeveralTokens'
        }      
    ],
    dictionaries = [
        {
            'dictionary_id': 'Word',
            'max_dictionary_size': '50000'
        }
    ],
    feature_calcers = [
        'BoW:top_tokens_count=10000'
    ]
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.2149103	test: 0.2178257	best: 0.2178257 (0)	total: 76.1ms	remaining: 6m 20s
100:	learn: 0.5528906	test: 0.5379696	best: 0.5383693 (99)	total: 3.37s	remaining: 2m 43s
200:	learn: 0.5934143	test: 0.5631495	best: 0.5633493 (197)	total: 6.26s	remaining: 2m 29s
300:	learn: 0.6169490	test: 0.5747402	best: 0.5747402 (294)	total: 9.16s	remaining: 2m 23s
400:	learn: 0.6332384	test: 0.5859313	best: 0.5861311 (399)	total: 12s	remaining: 2m 17s
500:	learn: 0.6493279	test: 0.5929257	best: 0.5933253 (498)	total: 14.9s	remaining: 2m 13s
600:	learn: 0.6607705	test: 0.5981215	best: 0.5983213 (596)	total: 17.8s	remaining: 2m 10s
700:	learn: 0.6719133	test: 0.6015188	best: 0.6015188 (700)	total: 20.7s	remaining: 2m 7s
800:	learn: 0.6819068	test: 0.6059153	best: 0.6061151 (792)	total: 23.6s	remaining: 2m 3s
900:	learn: 0.6906511	test: 0.6095124	best: 0.6101119 (892)	total: 26.6s	remaining: 2m 1s
1000:	learn: 0.6972468	test: 0.6099121	best: 0.6107114 (944)	total: 29.6s	remaining: 1m 58s
1100:	l

In [87]:
y_proba_cb = model.predict_proba(test_pool)
print("y_proba_cb",y_proba_cb)
y_pred = np.argmax(y_proba_cb, axis=1)
df_test.assign(predict = model.predict(test_pool))
# df_test['predict'] = model.predict(test_pool)

y_proba_cb [[1.06611788e-01 4.18916704e-02 5.54727313e-02 ... 1.79546882e-01
  1.96469313e-01 5.62750497e-02]
 [1.51593293e-02 4.13704391e-02 2.76633931e-02 ... 6.15628137e-02
  8.84245950e-03 3.61196641e-01]
 [7.33245749e-02 7.26666893e-02 7.73310370e-02 ... 8.61346448e-02
  4.09601043e-02 8.10271454e-02]
 ...
 [9.76413109e-04 9.89005409e-01 5.29836964e-04 ... 3.20403422e-04
  9.39938296e-05 1.26175818e-03]
 [1.98621235e-03 9.06424871e-01 4.23748397e-03 ... 5.32793868e-03
  3.78663646e-03 7.40034687e-04]
 [1.02016310e-01 8.85454761e-02 8.60886965e-02 ... 1.17434410e-01
  1.05155705e-01 6.05934034e-02]]


,Text,context_topic,predict
0,The eternal mystique of Goldman Sachs,7,9
2,I am such an IDIOT.,4,10
4,Something's watching me,0,4
5,Plantação de palma na Indonésia é uma dura rea...,0,8
6,The Milky Way Project: Probing Star Formation ...,9,9
...,...,...,...
3117,edit: 2020 has taken a few unfortunate turns.....,6,10
3118,---------------,3,0
3119,"That's huge, seriously. I bet it felt great, t...",1,1
3120,"Biologically, you'll sleep better in the dark,...",1,1


In [88]:
new_df_test = df_test.assign(predict = model.predict(test_pool))
new_df_test

,Text,context_topic,predict
0,The eternal mystique of Goldman Sachs,7,9
2,I am such an IDIOT.,4,10
4,Something's watching me,0,4
5,Plantação de palma na Indonésia é uma dura rea...,0,8
6,The Milky Way Project: Probing Star Formation ...,9,9
...,...,...,...
3117,edit: 2020 has taken a few unfortunate turns.....,6,10
3118,---------------,3,0
3119,"That's huge, seriously. I bet it felt great, t...",1,1
3120,"Biologically, you'll sleep better in the dark,...",1,1


In [89]:
acc = model.score(test_pool)
print(f"CatBoost model accuracy: {acc}")

CatBoost model accuracy: 0.7034904641957539


In [90]:
target_names = list(data_df['Context/Topic'].unique())

In [91]:
#1st try

print('\nClassification Report For Catboost')

print(classification_report(
    df_test['context_topic'].astype('category').cat.codes.values,
    y_pred, target_names=target_names))


Classification Report For Catboost
               precision    recall  f1-score   support

     Politics       0.58      0.76      0.66       245
         Love       0.76      0.75      0.75       300
Heavy Emotion       0.78      0.70      0.74       218
       Health       0.69      0.66      0.67       239
      Animals       0.64      0.61      0.62       295
      Science       0.74      0.68      0.71       187
         Joke       0.70      0.61      0.65       233
   Compliment       0.89      0.82      0.85       240
     Religion       0.75      0.71      0.73       318
         Self       0.60      0.73      0.66       221
    Education       0.68      0.71      0.70       283

     accuracy                           0.70      2779
    macro avg       0.71      0.70      0.70      2779
 weighted avg       0.71      0.70      0.70      2779

